In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.utils import shuffle

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

In [2]:

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


#  Loading scaled features

In [3]:
data_path = "./10ms_50rows/"

S1_x = np.load(data_path + "x_S1_scaled_XYZ_10ms_50lb.npy").astype(np.float32)
S1_y = np.load(data_path + "y_S1_scaled_XYZ_10ms_50lb.npy").astype(np.float32)

S2_x = np.load(data_path + "x_S2_scaled_XYZ_10ms_50lb.npy").astype(np.float32)
S2_y = np.load(data_path + "y_S2_scaled_XYZ_10ms_50lb.npy").astype(np.float32)

S3_x = np.load(data_path + "x_S3_scaled_XYZ_10ms_50lb.npy").astype(np.float32)
S3_y = np.load(data_path + "y_S3_scaled_XYZ_10ms_50lb.npy").astype(np.float32)

C1_x = np.load(data_path + "x_C1_scaled_XYZ_10ms_50lb.npy").astype(np.float32)
C1_y = np.load(data_path + "y_C1_scaled_XYZ_10ms_50lb.npy").astype(np.float32)

C2_x = np.load(data_path + "x_C2_scaled_XYZ_10ms_50lb.npy").astype(np.float32)
C2_y = np.load(data_path + "y_C2_scaled_XYZ_10ms_50lb.npy").astype(np.float32)

C3_x = np.load(data_path + "x_C3_scaled_XYZ_10ms_50lb.npy").astype(np.float32)
C3_y = np.load(data_path + "y_C3_scaled_XYZ_10ms_50lb.npy").astype(np.float32)

# Set up training and testing dats
# Train on S1 - C2, test on C3

X_train = [S1_x, S2_x, S3_x, C1_x, C2_x]
Y_train = [S1_y, S2_y, S3_y, C1_y, C2_y]

X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)

X_test = C3_x
Y_test = C3_y

In [4]:
S1_x

array([[[ 0.65048856, -0.29491472,  0.04537556],
        [ 0.6443313 , -0.30964693,  0.04340368],
        [ 0.6350953 , -0.3219238 ,  0.06706619],
        ...,
        [-5.4313917 ,  7.0589256 ,  1.8733039 ],
        [-5.346729  ,  7.7889895 ,  3.2023478 ],
        [-5.132762  ,  8.492863  ,  3.6046102 ]],

       [[ 1.1677023 ,  0.10285561,  0.03748805],
        [ 1.1738597 ,  0.09548949,  0.02467087],
        [ 1.1461518 ,  0.09385257,  0.01579743],
        ...,
        [-3.5133903 ,  4.6092825 ,  3.938843  ],
        [-3.120862  ,  4.965311  ,  4.443643  ],
        [-2.8576372 ,  5.409734  ,  5.0677414 ]],

       [[ 0.9829832 ,  0.08484954,  0.34510058],
        [ 0.9860618 ,  0.06193274,  0.31848025],
        [ 1.0183877 ,  0.04310821,  0.32143807],
        ...,
        [ 6.3875594 ,  3.4699895 ,  8.507678  ],
        [ 7.0002117 ,  3.2440953 ,  9.639534  ],
        [ 7.515886  ,  2.9838257 , 10.531808  ]],

       ...,

       [[ 1.3277924 ,  0.31729147, -0.5708354 ],
        [ 1

In [ ]:
def eval_lstm_cplx(X_train, Y_train, X_test, Y_test, time_steps, n_features, params):
  # Build the Model
  model = Sequential()
  model.add(LSTM(params['HL_1'], activation=tf.nn.relu, input_shape=(time_steps, n_features), return_sequences=True))
  model.add(Dropout(params['drop1']))
  model.add(BatchNormalization())
  model.add(LSTM(params['HL_2'], activation=tf.nn.relu))
  model.add(Dropout(params['drop2']))
  model.add(BatchNormalization())
#   model.add(Dense(10, activation = tf.nn.relu))
  model.add(Dense(1, activation = tf.nn.sigmoid))
    
#   opt = keras.optimizers.Adam(learning_rate=0.001)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience = params['patience'], verbose=1)
  model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = 'nadam', metrics=['accuracy'])

  # X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)
  
  # Implement class weights
  cw = {
      0: params['w0'],
      1: params['w1']
  }
  model.summary()
  # Train the model
  history = model.fit(X_train, Y_train, epochs = params['epochs'], batch_size = params['batch'], class_weight=cw, callbacks=[callback])

  # Predict on data
  Y_pred = model.predict(X_test)
  Y_pred = np.round(Y_pred, 0)
  np.where(Y_pred == 1)
  cm = confusion_matrix(Y_test, Y_pred)
  print("CONFUSION MATRIX")
  print(cm)
  print("CLASSIFICATION REPORT")
  print(classification_report(Y_test, Y_pred, target_names=["non_fall", "fall"]))
    
  return history, cm, model


params = { # With Feature Scaling
    'drop1' : 0.4,
    'drop2': 0.3,
    'w1' : 750.,
    'w0' : 1.,
    'batch' : 7500,
    'epochs' : 5,
    'patience' : 5,
    'HL_1' : 100,
    'HL_2' : 50,
}

In [5]:
S1_x.shape

(63099, 50, 3)

In [6]:
S2_x.shape

(57348, 50, 3)